In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/vlsp-restaurant/2-VLSP2018-SA-Restaurant-dev.csv
/kaggle/input/vlsp-restaurant/1-VLSP2018-SA-Restaurant-train.csv
/kaggle/input/vlsp-restaurant/3-VLSP2018-SA-Restaurant-test.csv
/kaggle/input/vlsp-hotel/3-VLSP2018-SA-Hotel-test.csv
/kaggle/input/vlsp-hotel/2-VLSP2018-SA-Hotel-dev.csv
/kaggle/input/vlsp-hotel/1-VLSP2018-SA-Hotel-train.csv
/kaggle/input/vrbp-data/full_data.csv


In [2]:
vrbp = pd.read_csv("/kaggle/input/vrbp-data/full_data.csv", encoding = "UTF-8")
vrbp.head(5)

,data,stayingpower,texture,smell,price,others,colour,shipping,packing
0,Công dụng: tốt\r\nKết cấu: đẹp\r\nĐộ bền màu: ...,positive,positive,NaN,NaN,NaN,NaN,NaN,NaN
1,Công dụng: son môi\r\nKết cấu: khô\r\nĐộ bền m...,positive,positive,NaN,NaN,NaN,NaN,NaN,NaN
2,"Son mịn, mùi thơm nhẹ, lâu trôi.\r\nVideo+ hìn...",positive,positive,positive,NaN,NaN,NaN,NaN,NaN
3,Công dụng: đánh son\r\nKết cấu: Đóng gói cẩn t...,positive,NaN,NaN,positive,NaN,NaN,negative,NaN
4,Công dụng: tốt\r\nKết cấu: tốt\r\nĐộ bền màu: ...,positive,positive,NaN,NaN,NaN,neutral,NaN,NaN


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

class ABSADataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):  # Reduced default
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        encoding = self.tokenizer(
            text, truncation=True, padding='max_length',
            max_length=self.max_length, return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

# =============================================================================
# 🎯 SINGLE-TASK LEARNING (STL) MODEL
# =============================================================================
class STLModel(nn.Module):
    """Single-Task Learning: Một model cho một aspect duy nhất"""
    
    def __init__(self, model_name, num_classes, dropout_rate=0.3):
        super(STLModel, self).__init__()
        self.transformer = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(dropout_rate)
        self.classifier = nn.Linear(self.transformer.config.hidden_size, num_classes)
        
    def forward(self, input_ids, attention_mask):
        outputs = self.transformer(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        output = self.dropout(pooled_output)
        logits = self.classifier(output)
        return logits

# =============================================================================
# 🤝 MULTI-TASK LEARNING (MTL) MODEL  
# =============================================================================
class MTLModel(nn.Module):
    """Multi-Task Learning: Một model cho nhiều aspects cùng lúc"""
    
    def __init__(self, model_name, aspect_classes, dropout_rate=0.3):
        super(MTLModel, self).__init__()
        # Shared transformer encoder
        self.transformer = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(dropout_rate)
        
        # Task-specific classifiers cho mỗi aspect
        self.classifiers = nn.ModuleDict()
        for aspect, num_classes in aspect_classes.items():
            self.classifiers[aspect] = nn.Linear(
                self.transformer.config.hidden_size, num_classes
            )
    
    def forward(self, input_ids, attention_mask):
        # Shared feature extraction
        outputs = self.transformer(input_ids=input_ids, attention_mask=attention_mask)
        shared_features = self.dropout(outputs.pooler_output)
        
        # Task-specific predictions
        logits = {}
        for aspect in self.classifiers:
            logits[aspect] = self.classifiers[aspect](shared_features)
        
        return logits

class STL_vs_MTL_Comparison:
    """So sánh STL vs MTL trên multi-domain ABSA"""
    
    def __init__(self, model_name="vinai/phobert-base", batch_size=2, max_length=128, force_cpu=False):
        self.model_name = model_name
        self.batch_size = batch_size  # Reduced from 8 to 2
        self.max_length = max_length  # Reduced from 256 to 128
        
        # Device selection with memory check
        if force_cpu:
            self.device = torch.device('cpu')
            print("🖥️  Forcing CPU usage")
        else:
            self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        
        print(f"🚀 STL vs MTL Comparison System")
        print(f"📱 Device: {self.device}")
        print(f"🤖 Model: {model_name}")
        print(f"🔧 Batch size: {batch_size}, Max length: {max_length}")
        
        # Clear GPU cache if available
        if torch.cuda.is_available() and not force_cpu:
            torch.cuda.empty_cache()
            total_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
            print(f"💾 GPU Memory: {total_memory:.1f}GB total")
            
            # Check available memory
            available_memory = (torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_allocated()) / 1e9
            print(f"💾 Available GPU Memory: {available_memory:.1f}GB")
            
            if available_memory < 2.0:  # Less than 2GB available
                print("⚠️  Low GPU memory detected. Consider using force_cpu=True")
        
        # Storage
        self.datasets = {}
        self.stl_models = {}  # STL: Một model per aspect
        self.mtl_models = {}  # MTL: Một model per domain với nhiều aspects
        self.results = {'STL': {}, 'MTL': {}}
    
    def switch_to_cpu(self):
        """Switch to CPU mode if GPU runs out of memory"""
        self.device = torch.device('cpu')
        print("🔄 Switched to CPU mode due to memory constraints")
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
    
    def clear_memory(self):
        """Clear GPU memory cache"""
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
    
    def load_domain_data(self, domain_name, file_path_or_data):
        """Load dữ liệu cho một domain"""
        print(f"\n📊 Loading {domain_name} domain...")
        
        if domain_name == 'cosmetic':
            return self._load_cosmetic_data(file_path_or_data)
        else:
            return self._load_vlsp_data(file_path_or_data, domain_name)
    
    def _load_cosmetic_data(self, file_path):
        """Load cosmetic dataset"""
        df = pd.read_csv(file_path, encoding='latin-1')
        text_col = 'data'
        aspects = [col for col in df.columns if col != 'data']
        
        df_clean = df.dropna(subset=[text_col])
        aspect_counts = df_clean[aspects].notna().sum(axis=1)
        df_clean = df_clean[aspect_counts > 0]
        
        domain_data = {}
        for aspect in aspects:
            aspect_df = df_clean.dropna(subset=[aspect])
            if len(aspect_df) > 100:  # Đủ dữ liệu
                X = aspect_df[text_col].values
                y = aspect_df[aspect].values
                
                encoder = LabelEncoder()
                y_encoded = encoder.fit_transform(y)
                
                # Check if each class has at least 2 samples
                unique, counts = np.unique(y_encoded, return_counts=True)
                min_class_count = np.min(counts)
                
                if len(encoder.classes_) >= 2 and min_class_count >= 2:
                    domain_data[aspect] = {
                        'X': X, 'y': y_encoded, 
                        'encoder': encoder, 'classes': encoder.classes_
                    }
                    print(f"  ✅ {aspect}: {len(X)} samples, {len(encoder.classes_)} classes, min_class={min_class_count}")
                else:
                    print(f"  ❌ {aspect}: Skipped (classes={len(encoder.classes_)}, min_class_count={min_class_count})")
        
        self.datasets['cosmetic'] = domain_data
        return domain_data
    
    def _load_vlsp_data(self, data_path, domain_name):
        """Load VLSP Hotel/Restaurant data"""
        train_df = pd.read_csv(f"{data_path}/1-VLSP2018-SA-{domain_name.title()}-train.csv")
        dev_df = pd.read_csv(f"{data_path}/2-VLSP2018-SA-{domain_name.title()}-dev.csv")
        
        full_df = pd.concat([train_df, dev_df], ignore_index=True)
        text_col = 'Review'
        aspect_cols = [col for col in full_df.columns if col != 'Review']
        
        domain_data = {}
        for aspect in aspect_cols[:5]:  # Limit cho demo
            aspect_mask = full_df[aspect] != 0
            if aspect_mask.sum() > 100:
                X = full_df[aspect_mask][text_col].values
                y = full_df[aspect_mask][aspect].values
                
                unique_labels = sorted(np.unique(y))
                label_mapping = {label: idx for idx, label in enumerate(unique_labels)}
                y_encoded = np.array([label_mapping[label] for label in y])
                
                # Check if each class has at least 2 samples
                unique, counts = np.unique(y_encoded, return_counts=True)
                min_class_count = np.min(counts)
                
                if len(unique_labels) >= 2 and min_class_count >= 2:
                    encoder = LabelEncoder()
                    encoder.classes_ = np.array([f"class_{i}" for i in range(len(unique_labels))])
                    
                    domain_data[aspect] = {
                        'X': X, 'y': y_encoded,
                        'encoder': encoder, 'classes': encoder.classes_
                    }
                    print(f"  ✅ {aspect}: {len(X)} samples, {len(unique_labels)} classes, min_class={min_class_count}")
                else:
                    print(f"  ❌ {aspect}: Skipped (classes={len(unique_labels)}, min_class_count={min_class_count})")
        
        self.datasets[domain_name] = domain_data
        return domain_data
    
    # =========================================================================
    # 🎯 SINGLE-TASK LEARNING IMPLEMENTATION
    # =========================================================================
    def train_stl_models(self, domain_name, epochs=2):
        """Train STL: Một model riêng cho mỗi aspect"""
        print(f"\n🎯 TRAINING STL MODELS for {domain_name.upper()}")
        print("=" * 60)
        
        domain_data = self.datasets[domain_name]
        stl_results = {}
        
        for aspect, data in domain_data.items():
            print(f"\n📈 STL Training: {aspect}")
            
            # Clear memory before training each model
            self.clear_memory()
            
            X, y = data['X'], data['y']
            num_classes = len(data['classes'])
            
            # Split data
            # Check if all classes have at least 2 samples for stratification
            unique, counts = np.unique(y, return_counts=True)
            can_stratify = np.all(counts >= 2)
            
            if can_stratify:
                X_train, X_val, y_train, y_val = train_test_split(
                    X, y, test_size=0.2, random_state=42, stratify=y
                )
            else:
                print(f"    ⚠️  Some classes have <2 samples, skipping stratification")
                X_train, X_val, y_train, y_val = train_test_split(
                    X, y, test_size=0.2, random_state=42
                )
            
            # Create datasets with optimized max_length
            train_dataset = ABSADataset(X_train, y_train, self.tokenizer, self.max_length)
            val_dataset = ABSADataset(X_val, y_val, self.tokenizer, self.max_length)
            
            train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=self.batch_size)
            
            # Create STL model - MỖI ASPECT CÓ MODEL RIÊNG
            model = STLModel(self.model_name, num_classes).to(self.device)
            optimizer = optim.AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
            criterion = nn.CrossEntropyLoss()
            
            # Gradient accumulation setup
            accumulation_steps = 2  # Effective batch size = batch_size * accumulation_steps
            
            # Training loop
            model.train()
            for epoch in range(epochs):
                total_loss = 0
                optimizer.zero_grad()
                
                for batch_idx, batch in enumerate(tqdm(train_loader, desc=f"STL Epoch {epoch+1}")):
                    input_ids = batch['input_ids'].to(self.device)
                    attention_mask = batch['attention_mask'].to(self.device)
                    labels = batch['labels'].to(self.device)
                    
                    logits = model(input_ids, attention_mask)
                    loss = criterion(logits, labels)
                    
                    # Scale loss for gradient accumulation
                    loss = loss / accumulation_steps
                    loss.backward()
                    
                    # Update weights every accumulation_steps
                    if (batch_idx + 1) % accumulation_steps == 0:
                        optimizer.step()
                        optimizer.zero_grad()
                        
                        # Clear cache periodically
                        if (batch_idx + 1) % (accumulation_steps * 4) == 0:
                            self.clear_memory()
                    
                    total_loss += loss.item() * accumulation_steps
                    
                    # Delete batch tensors to free memory
                    del input_ids, attention_mask, labels, logits, loss
                
                # Final optimizer step if there are remaining gradients
                optimizer.step()
                optimizer.zero_grad()
                self.clear_memory()
                
                print(f"    Epoch {epoch+1} Average Loss: {total_loss/len(train_loader):.4f}")
            
            # Evaluation
            model.eval()
            all_preds, all_labels = [], []
            with torch.no_grad():
                for batch in val_loader:
                    input_ids = batch['input_ids'].to(self.device)
                    attention_mask = batch['attention_mask'].to(self.device)
                    labels = batch['labels'].to(self.device)
                    
                    logits = model(input_ids, attention_mask)
                    preds = torch.argmax(logits, dim=1)
                    
                    all_preds.extend(preds.cpu().numpy())
                    all_labels.extend(labels.cpu().numpy())
                    
                    # Clear batch tensors
                    del input_ids, attention_mask, labels, logits, preds
            
            self.clear_memory()
            
            accuracy = accuracy_score(all_labels, all_preds)
            f1 = f1_score(all_labels, all_preds, average='weighted')
            
            print(f"  ✅ STL {aspect}: Accuracy={accuracy:.3f}, F1={f1:.3f}")
            
            # Store results and move model to CPU to save GPU memory
            model_cpu = model.cpu()
            self.stl_models[f"{domain_name}_{aspect}"] = model_cpu
            stl_results[aspect] = {'accuracy': accuracy, 'f1_score': f1}
            
            # Clear GPU memory
            del model
            self.clear_memory()
        
        self.results['STL'][domain_name] = stl_results
        return stl_results
    
    # =========================================================================
    # 🤝 MULTI-TASK LEARNING IMPLEMENTATION  
    # =========================================================================
    def train_mtl_model(self, domain_name, epochs=2):
        """Train MTL: Một model cho tất cả aspects trong domain"""
        print(f"\n🤝 TRAINING MTL MODEL for {domain_name.upper()}")
        print("=" * 60)
        
        # Clear memory before training
        self.clear_memory()
        
        domain_data = self.datasets[domain_name]
        
        # Prepare MTL data - TẤT CẢ ASPECTS CÙNG LÚC
        all_texts = []
        all_labels = {}
        aspect_classes = {}
        
        # Get all unique texts
        unique_texts = set()
        for aspect, data in domain_data.items():
            unique_texts.update(data['X'])
        
        all_texts = list(unique_texts)
        
        # Create labels for each aspect
        for aspect, data in domain_data.items():
            aspect_classes[aspect] = len(data['classes'])
            
            # Map texts to labels
            text_to_label = dict(zip(data['X'], data['y']))
            labels = []
            for text in all_texts:
                if text in text_to_label:
                    labels.append(text_to_label[text])
                else:
                    labels.append(-1)  # No label cho aspect này
            
            all_labels[aspect] = np.array(labels)
        
        print(f"  📊 MTL Training: {len(all_texts)} samples, {len(aspect_classes)} aspects")
        
        # Split data
        indices = np.arange(len(all_texts))
        train_idx, val_idx = train_test_split(indices, test_size=0.2, random_state=42)
        
        # Create MTL model - MỘT MODEL CHO TẤT CẢ ASPECTS
        model = MTLModel(self.model_name, aspect_classes).to(self.device)
        optimizer = optim.AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
        criterion = nn.CrossEntropyLoss(ignore_index=-1)  # Ignore missing labels
        
        # Gradient accumulation setup
        accumulation_steps = 2
        
        # Training loop
        model.train()
        for epoch in range(epochs):
            total_loss = 0
            num_batches = 0
            optimizer.zero_grad()
            
            batch_count = 0
            for i in tqdm(range(0, len(train_idx), self.batch_size), desc=f"MTL Epoch {epoch+1}"):
                batch_indices = train_idx[i:i+self.batch_size]
                batch_texts = [all_texts[idx] for idx in batch_indices]
                
                # Tokenize batch
                encoding = self.tokenizer(
                    batch_texts, truncation=True, padding='max_length',
                    max_length=self.max_length, return_tensors='pt'
                )
                
                input_ids = encoding['input_ids'].to(self.device)
                attention_mask = encoding['attention_mask'].to(self.device)
                
                # Forward pass
                logits = model(input_ids, attention_mask)
                
                # Calculate loss cho tất cả aspects
                total_batch_loss = 0
                batch_labels_list = []  # Store all batch_labels for deletion
                
                for aspect in aspect_classes:
                    batch_labels = torch.tensor(
                        all_labels[aspect][batch_indices], dtype=torch.long
                    ).to(self.device)
                    batch_labels_list.append(batch_labels)
                    
                    aspect_loss = criterion(logits[aspect], batch_labels)
                    total_batch_loss += aspect_loss
                
                # Scale loss for gradient accumulation
                total_batch_loss = total_batch_loss / accumulation_steps
                total_batch_loss.backward()
                
                batch_count += 1
                
                # Update weights every accumulation_steps
                if batch_count % accumulation_steps == 0:
                    optimizer.step()
                    optimizer.zero_grad()
                    
                    # Clear cache periodically
                    if batch_count % (accumulation_steps * 4) == 0:
                        self.clear_memory()
                
                total_loss += total_batch_loss.item() * accumulation_steps
                num_batches += 1
                
                # Delete batch tensors to free memory
                del input_ids, attention_mask, logits, total_batch_loss, encoding
                for batch_labels in batch_labels_list:
                    del batch_labels
                del batch_labels_list
            
            # Final optimizer step
            optimizer.step()
            optimizer.zero_grad()
            self.clear_memory()
            
            print(f"    Epoch {epoch+1} Average Loss: {total_loss/num_batches:.4f}")
        
        # Evaluation
        model.eval()
        mtl_results = {}
        
        for aspect in aspect_classes:
            all_preds, all_labels_aspect = [], []
            
            with torch.no_grad():
                for i in range(0, len(val_idx), self.batch_size):
                    batch_indices = val_idx[i:i+self.batch_size]
                    batch_texts = [all_texts[idx] for idx in batch_indices]
                    
                    encoding = self.tokenizer(
                        batch_texts, truncation=True, padding='max_length',
                        max_length=self.max_length, return_tensors='pt'
                    )
                    
                    input_ids = encoding['input_ids'].to(self.device)
                    attention_mask = encoding['attention_mask'].to(self.device)
                    
                    logits = model(input_ids, attention_mask)
                    preds = torch.argmax(logits[aspect], dim=1)
                    
                    all_preds.extend(preds.cpu().numpy())
                    all_labels_aspect.extend(all_labels[aspect][batch_indices])
                    
                    # Clear batch tensors
                    del input_ids, attention_mask, logits, preds, encoding
            
            # Filter valid labels
            valid_mask = np.array(all_labels_aspect) != -1
            if valid_mask.sum() > 0:
                valid_preds = np.array(all_preds)[valid_mask]
                valid_labels = np.array(all_labels_aspect)[valid_mask]
                
                accuracy = accuracy_score(valid_labels, valid_preds)
                f1 = f1_score(valid_labels, valid_preds, average='weighted')
                
                print(f"  ✅ MTL {aspect}: Accuracy={accuracy:.3f}, F1={f1:.3f}")
                mtl_results[aspect] = {'accuracy': accuracy, 'f1_score': f1}
        
        # Move model to CPU to save GPU memory
        model_cpu = model.cpu()
        self.mtl_models[domain_name] = model_cpu
        self.results['MTL'][domain_name] = mtl_results
        
        # Clear GPU memory
        del model
        self.clear_memory()
        
        return mtl_results
    
    def comprehensive_comparison(self):
        """So sánh tổng thể STL vs MTL"""
        print("\n" + "="*80)
        print("📊 COMPREHENSIVE STL vs MTL COMPARISON")
        print("="*80)
        
        for domain in self.results['STL']:
            print(f"\n🏢 {domain.upper()} DOMAIN:")
            print("-" * 50)
            
            stl_results = self.results['STL'][domain]
            mtl_results = self.results['MTL'].get(domain, {})
            
            stl_accs, stl_f1s = [], []
            mtl_accs, mtl_f1s = [], []
            
            for aspect in stl_results:
                stl_acc = stl_results[aspect]['accuracy']
                stl_f1 = stl_results[aspect]['f1_score']
                
                stl_accs.append(stl_acc)
                stl_f1s.append(stl_f1)
                
                if aspect in mtl_results:
                    mtl_acc = mtl_results[aspect]['accuracy']
                    mtl_f1 = mtl_results[aspect]['f1_score']
                    
                    mtl_accs.append(mtl_acc)
                    mtl_f1s.append(mtl_f1)
                    
                    print(f"  {aspect}:")
                    print(f"    STL: Acc={stl_acc:.3f}, F1={stl_f1:.3f}")
                    print(f"    MTL: Acc={mtl_acc:.3f}, F1={mtl_f1:.3f}")
                    print(f"    Diff: Acc={mtl_acc-stl_acc:+.3f}, F1={mtl_f1-stl_f1:+.3f}")
            
            if stl_accs and mtl_accs:
                print(f"\n  📈 DOMAIN AVERAGE:")
                print(f"    STL: Acc={np.mean(stl_accs):.3f}, F1={np.mean(stl_f1s):.3f}")
                print(f"    MTL: Acc={np.mean(mtl_accs):.3f}, F1={np.mean(mtl_f1s):.3f}")
                print(f"    MTL Improvement: Acc={np.mean(mtl_accs)-np.mean(stl_accs):+.3f}, F1={np.mean(mtl_f1s)-np.mean(stl_f1s):+.3f}")
        
        # Key insights
        print(f"\n🔍 KEY INSIGHTS:")
        print(f"  🎯 STL: Mỗi aspect có model riêng biệt")
        print(f"  🤝 MTL: Một model chia sẻ cho tất cả aspects")
        print(f"  💡 MTL có thể tận dụng shared knowledge giữa aspects")
        print(f"  ⚡ STL có thể tập trung tốt hơn cho từng aspect cụ thể")

def main():
    print("🚀 STL vs MTL COMPREHENSIVE COMPARISON")
    print("="*80)
    
    # Try GPU first, fallback to CPU if memory issues
    try:
        # Initialize comparison system with optimized settings
        comparison = STL_vs_MTL_Comparison(
            model_name="vinai/phobert-base",
            batch_size=1,  # Further reduced to 1
            max_length=96  # Further reduced to 96
        )
        
        # Load datasets
        print("\n📚 LOADING DATASETS...")
        
        # Load datasets with error handling
        domains_loaded = []
        
        try:
            cosmetic_data = comparison.load_domain_data('cosmetic', '/kaggle/input/vrbp-data/full_data.csv')
            domains_loaded.append('cosmetic')
        except Exception as e:
            print(f"❌ Failed to load cosmetic data: {e}")
        
        try:
            hotel_data = comparison.load_domain_data('hotel', '/kaggle/input/vlsp-hotel')
            domains_loaded.append('hotel')
        except Exception as e:
            print(f"❌ Failed to load hotel data: {e}")
        
        try:
            restaurant_data = comparison.load_domain_data('restaurant', '/kaggle/input/vlsp-restaurant')
            domains_loaded.append('restaurant')
        except Exception as e:
            print(f"❌ Failed to load restaurant data: {e}")
        
        # STL vs MTL Training for loaded domains only
        for domain in domains_loaded:
            if domain in comparison.datasets and len(comparison.datasets[domain]) > 0:
                print(f"\n{'='*80}")
                print(f"🔄 TRAINING {domain.upper()} MODELS")
                print(f"{'='*80}")
                
                try:
                    # Train STL models
                    print(f"💾 GPU Memory before STL: {torch.cuda.memory_allocated()/1e9:.2f}GB" if torch.cuda.is_available() else "Using CPU")
                    comparison.train_stl_models(domain, epochs=1)
                    
                    # Train MTL model  
                    print(f"💾 GPU Memory before MTL: {torch.cuda.memory_allocated()/1e9:.2f}GB" if torch.cuda.is_available() else "Using CPU")
                    comparison.train_mtl_model(domain, epochs=1)
                    
                except RuntimeError as e:
                    if "out of memory" in str(e):
                        print(f"❌ GPU Memory exhausted for {domain}. Trying with CPU...")
                        comparison.clear_memory()
                        
                        # Create new comparison instance with CPU
                        cpu_comparison = STL_vs_MTL_Comparison(
                            model_name="vinai/phobert-base",
                            batch_size=2,  # Can use larger batch on CPU
                            max_length=96,
                            force_cpu=True
                        )
                        cpu_comparison.datasets[domain] = comparison.datasets[domain]
                        
                        try:
                            cpu_comparison.train_stl_models(domain, epochs=1)
                            cpu_comparison.train_mtl_model(domain, epochs=1)
                            
                            # Copy results back
                            comparison.results['STL'][domain] = cpu_comparison.results['STL'][domain]
                            comparison.results['MTL'][domain] = cpu_comparison.results['MTL'][domain]
                            
                            print(f"✅ {domain} completed using CPU")
                        except Exception as cpu_e:
                            print(f"❌ Failed to train {domain} even on CPU: {cpu_e}")
                    else:
                        raise e
            else:
                print(f"⏭️  Skipping {domain} - no valid data loaded")
        
        # Final comparison
        comparison.comprehensive_comparison()
        
        print("\n✅ STL vs MTL Comparison Complete!")
        
    except RuntimeError as e:
        if "out of memory" in str(e):
            print("\n❌ GPU out of memory. Try these solutions:")
            print("   1. Use CPU mode: add force_cpu=True")
            print("   2. Reduce batch_size to 1")
            print("   3. Reduce max_length to 64 or 32")
            print("   4. Use smaller models:")
            print("      - 'distilbert-base-multilingual-cased' (smaller BERT)")
            print("      - 'xlm-roberta-base' (multilingual but smaller)")
            print("      - 'bert-base-multilingual-cased' (alternative)")
            print("\n   Example with smaller model:")
            print("   comparison = STL_vs_MTL_Comparison(")
            print("       model_name='distilbert-base-multilingual-cased',")
            print("       batch_size=1, max_length=64, force_cpu=True)")
        else:
            raise e
    except Exception as e:
        print(f"\n❌ Error occurred: {e}")
        print("Please check your data files and paths.")

# Alternative lightweight comparison for very limited memory
def lightweight_comparison():
    """Ultra-lightweight version for very limited memory"""
    print("🚀 LIGHTWEIGHT STL vs MTL COMPARISON")
    print("="*80)
    
    comparison = STL_vs_MTL_Comparison(
        model_name="distilbert-base-multilingual-cased",  # Smaller model
        batch_size=1,
        max_length=64,  # Very short sequences
        force_cpu=True  # Force CPU usage
    )
    
    # Load only cosmetic data for demo
    try:
        cosmetic_data = comparison.load_domain_data('cosmetic', '/kaggle/input/vrbp-data/full_data.csv')
        if 'cosmetic' in comparison.datasets:
            comparison.train_stl_models('cosmetic', epochs=1)
            comparison.train_mtl_model('cosmetic', epochs=1)
            comparison.comprehensive_comparison()
    except Exception as e:
        print(f"❌ Error in lightweight comparison: {e}")

if __name__ == "__main__":
    main() 

🚀 STL vs MTL COMPREHENSIVE COMPARISON


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

    Epoch 1 Average Loss: 0.4501
  ✅ STL DRINKS#STYLE&OPTIONS: Accuracy=0.913, F1=0.872

📈 STL Training: FOOD#PRICES


STL Epoch 1: 100%|██████████| 1700/1700 [01:36<00:00, 17.71it/s]


    Epoch 1 Average Loss: 0.8656
  ✅ STL FOOD#PRICES: Accuracy=0.784, F1=0.790
💾 GPU Memory before MTL: 0.02GB

🤝 TRAINING MTL MODEL for RESTAURANT
  📊 MTL Training: 2601 samples, 5 aspects


MTL Epoch 1: 100%|██████████| 2080/2080 [01:59<00:00, 17.35it/s]


    Epoch 1 Average Loss: nan
  ✅ MTL AMBIENCE#GENERAL: Accuracy=0.678, F1=0.554
  ✅ MTL DRINKS#PRICES: Accuracy=0.455, F1=0.284
  ✅ MTL DRINKS#QUALITY: Accuracy=0.767, F1=0.666
  ✅ MTL DRINKS#STYLE&OPTIONS: Accuracy=0.871, F1=0.811
  ✅ MTL FOOD#PRICES: Accuracy=0.724, F1=0.696

📊 COMPREHENSIVE STL vs MTL COMPARISON

🏢 COSMETIC DOMAIN:
--------------------------------------------------
  stayingpower:
    STL: Acc=0.635, F1=0.581
    MTL: Acc=0.542, F1=0.381
    Diff: Acc=-0.093, F1=-0.199
  texture:
    STL: Acc=0.724, F1=0.656
    MTL: Acc=0.705, F1=0.583
    Diff: Acc=-0.019, F1=-0.073
  smell:
    STL: Acc=0.804, F1=0.717
    MTL: Acc=0.816, F1=0.734
    Diff: Acc=+0.012, F1=+0.016
  price:
    STL: Acc=0.985, F1=0.977
    MTL: Acc=0.978, F1=0.967
    Diff: Acc=-0.007, F1=-0.010
  colour:
    STL: Acc=0.840, F1=0.767
    MTL: Acc=0.828, F1=0.750
    Diff: Acc=-0.012, F1=-0.017
  shipping:
    STL: Acc=0.858, F1=0.830
    MTL: Acc=0.664, F1=0.529
    Diff: Acc=-0.195, F1=-0.301
  pa